In [0]:
#bronze
import os

# Acessar as variáveis de ambiente
storage_account_name = os.getenv("AZURE_STORAGE_ACCOUNT_NAME")
storage_account_key = os.getenv("AZURE_STORAGE_KEY")

# Montar o Blob Storage ou Data Lake
if "/mnt/bronze" not in [mount.mountPoint for mount in dbutils.fs.mounts()]:
    dbutils.fs.mount(
        source = f"wasbs://bronze@brewerydatalake.blob.core.windows.net/",
        mount_point = "/mnt/bronze",
        extra_configs = {f"fs.azure.account.key.brewerydatalake.blob.core.windows.net": storage_account_key}
    )
else:
    print("O diretório já está montado.")


O diretório já está montado.


In [0]:
display(dbutils.fs.ls("/mnt/bronze"))


path,name,size,modificationTime
dbfs:/mnt/bronze/breweries_raw.json,breweries_raw.json,22375,1729413585000


In [0]:
# silver
import os

# Acessar as variáveis de ambiente
storage_account_name = os.getenv("AZURE_STORAGE_ACCOUNT_NAME")
storage_account_key = os.getenv("AZURE_STORAGE_KEY")

# Montar o Blob Storage ou Data Lake
if "/mnt/silver" not in [mount.mountPoint for mount in dbutils.fs.mounts()]:
    dbutils.fs.mount(
        source = f"wasbs://silver@brewerydatalake.blob.core.windows.net/",
        mount_point = "/mnt/silver",
        extra_configs = {f"fs.azure.account.key.brewerydatalake.blob.core.windows.net": storage_account_key}
    )
else:
    print("O diretório já está montado.")

O diretório já está montado.


In [0]:
display(dbutils.fs.ls("/mnt/silver"))


path,name,size,modificationTime
dbfs:/mnt/silver/breweries/,breweries/,0,0


In [0]:
# gold

import os

# Acessar as variáveis de ambiente
storage_account_name = os.getenv("AZURE_STORAGE_ACCOUNT_NAME")
storage_account_key = os.getenv("AZURE_STORAGE_KEY")

# Montar o Blob Storage ou Data Lake
if "/mnt/gold" not in [mount.mountPoint for mount in dbutils.fs.mounts()]:
    dbutils.fs.mount(
        source = f"wasbs://gold@brewerydatalake.blob.core.windows.net/",
        mount_point = "/mnt/gold",
        extra_configs = {f"fs.azure.account.key.brewerydatalake.blob.core.windows.net": storage_account_key}
    )
else:
    print("O diretório já está montado.")



O diretório já está montado.


In [0]:
import requests
import json

# Requisição para a API Open Brewery DB
url = "https://api.openbrewerydb.org/breweries"
response = requests.get(url)
data = response.json()

# Salvar os dados brutos como JSON na camada bronze
with open('/dbfs/mnt/bronze/breweries_raw.json', 'w') as f:
    json.dump(data, f)


In [0]:
# Ler os dados da camada bronze
df_bronze = spark.read.json("/mnt/bronze/breweries_raw.json")

# Aplicar transformações
df_silver = df_bronze.filter(df_bronze["brewery_type"].isNotNull()) \
                     .withColumn("latitude", df_bronze["latitude"].cast("double")) \
                     .withColumn("longitude", df_bronze["longitude"].cast("double"))

# Dropar a tabela existente se necessário
spark.sql("DROP TABLE IF EXISTS pipelinedatabricks.brewery.silver_breweries")

# Salvar em formato Delta e particionar por localização (estado ou cidade)
df_silver.write.format("delta").mode("overwrite").partitionBy("state_province").save("/mnt/silver/breweries")

# Criar a tabela silver no Databricks
df_silver.write.format("delta").mode("overwrite").saveAsTable("pipelinedatabricks.brewery.silver_breweries")



In [0]:
# Ler os dados da camada silver
df_silver = spark.read.format("delta").load("/mnt/silver/breweries")

# Realizar a agregação (contagem de cervejarias por tipo e localização)
df_gold = df_silver.groupBy("state_province", "brewery_type").count()

# Dropar a tabela existente se necessário
spark.sql("DROP TABLE IF EXISTS pipelinedatabricks.brewery.gold_breweries")

# Salvar os dados agregados na camada gold
df_gold.write.format("delta").mode("overwrite").save("/mnt/gold/breweries")

# Criar a tabela gold no Databricks
df_gold.write.format("delta").mode("overwrite").saveAsTable("pipelinedatabricks.brewery.gold_breweries")






In [0]:
# Ler os dados da camada bronze
df_bronze = spark.read.json("/mnt/bronze/breweries_raw.json")

# Exibir uma amostra dos dados
df_bronze.show(10)


+--------------------+---------+---------+------------+--------------+-------------+--------------------+-----------------+------------------+--------------------+----------+-----------+----------+--------------+--------------------+--------------------+
|           address_1|address_2|address_3|brewery_type|          city|      country|                  id|         latitude|         longitude|                name|     phone|postal_code|     state|state_province|              street|         website_url|
+--------------------+---------+---------+------------+--------------+-------------+--------------------+-----------------+------------------+--------------------+----------+-----------+----------+--------------+--------------------+--------------------+
|      1716 Topeka St|     NULL|     NULL|       micro|        Norman|United States|5128df48-79fc-4f0...|      35.25738891|      -97.46818222|    (405) Brewing Co|4058160490| 73069-8224|  Oklahoma|      Oklahoma|      1716 Topeka St|ht

In [0]:
# Ler os dados da camada silver
df_silver = spark.read.format("delta").load("/mnt/silver/breweries")

# Exibir uma amostra dos dados transformados
df_silver.show(10)


+--------------------+---------+---------+------------+----------------+-------------+--------------------+-----------+------------+--------------------+----------+-----------+----------+--------------+--------------------+--------------------+
|           address_1|address_2|address_3|brewery_type|            city|      country|                  id|   latitude|   longitude|                name|     phone|postal_code|     state|state_province|              street|         website_url|
+--------------------+---------+---------+------------+----------------+-------------+--------------------+-----------+------------+--------------------+----------+-----------+----------+--------------+--------------------+--------------------+
|           1501 E St|     NULL|     NULL|       large|       San Diego|United States|ef970757-fe42-416...|  32.714813| -117.129593|10 Barrel Brewing Co|6195782311| 92101-6618|California|    California|           1501 E St| http://10barrel.com|
| 1304 Scott St Ste 

In [0]:
# Ler os dados agregados da camada gold
df_gold = spark.read.format("delta").load("/mnt/gold/breweries")

# Exibir uma amostra dos dados agregados
df_gold.show(10)


+--------------+------------+-----+
|state_province|brewery_type|count|
+--------------+------------+-----+
|    California|       micro|    2|
|        Oregon|       large|    4|
|    California|       large|    1|
|      Colorado|     brewpub|    1|
| Massachusetts|       micro|    1|
|      Colorado|  proprietor|    1|
|       Indiana|       micro|    3|
|       Arizona|       micro|    3|
|      Colorado|       micro|    1|
|      Colorado|       large|    1|
+--------------+------------+-----+
only showing top 10 rows

